In [4]:
import requests
import pandas as pd

# API endpoint for Iowa Liquor Sales data in JSON format
url = "https://data.iowa.gov/resource/m3tr-qhgy.json"

# Set parameters to limit the data to 100 rows
params = {
    "$limit": 100  # Limit to the first 100 records
}

# Make a GET request to the API with query parameters
response = requests.get(url, params=params)

# Check if the request was successful (status code 200 means success)
if response.status_code == 200:
    data = response.json()  # Parse the JSON response
    
    # Convert the JSON data into a Pandas DataFrame
    df = pd.DataFrame(data)
    
    # Print the first 5 rows of the DataFrame
    print("First 5 rows of the DataFrame:")
    print(df.head())
else:
    print("Failed to retrieve data. Status code:", response.status_code)


First 5 rows of the DataFrame:
   invoice_line_no                     date store  \
0  INV-41268900020  2021-10-22T00:00:00.000  5751   
1  INV-41309800010  2021-10-25T00:00:00.000  2635   
2  INV-41325400013  2021-10-25T00:00:00.000  4269   
3  INV-41141800017  2021-10-19T00:00:00.000  2704   
4  INV-41308200068  2021-10-25T00:00:00.000  2512   

                                       name               address        city  \
0  CASEY'S GENERAL STORE #2346 / BURLINGTON        2630 SUMMER ST  BURLINGTON   
1                     HY-VEE #4 / DAVENPORT        4064 E 53RD ST   DAVENPORT   
2                       TIPTON FAMILY FOODS          610 CEDAR ST      TIPTON   
3               HY-VEE OELWEIN DOLLAR FRESH  1345 S FREDERICK AVE     OELWEIN   
4       HY-VEE WINE AND SPIRITS / IOWA CITY    1720 WATERFRONT DR   IOWA CITY   

  zipcode                                     store_location county_number  \
0   52601  {'type': 'Point', 'coordinates': [-91.117485, ...            29   
1   528

In [5]:
# 1. Standardize column names (remove special characters, replace spaces with underscores)
df.columns = df.columns.str.replace(r'[^a-zA-Z0-9_]', '', regex=True)
df.columns = df.columns.str.strip().str.lower()

# 2. Convert 'date' column to datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')  # 'errors=coerce' will turn invalid dates into NaT

# 3. Convert numeric columns to appropriate data types (e.g., 'sale_dollars', 'sale_bottles')
numeric_columns = ['sale_dollars', 'sale_bottles', 'sale_liters', 'sale_gallons']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# 4. Drop unnecessary columns (e.g., computed region columns)
df = df.drop(columns=[col for col in df.columns if col.startswith(':@')])

# 5. Handle missing values (optional, depending on your needs)
df = df.dropna(subset=['date'])  # Drop rows with missing 'date' values (for example)

# Fill missing values in numerical columns with zeros:
df[numeric_columns] = df[numeric_columns].fillna(0)

# 6. Check cleaned data
print(df.head())


   invoice_line_no       date store                                      name  \
0  INV-41268900020 2021-10-22  5751  CASEY'S GENERAL STORE #2346 / BURLINGTON   
1  INV-41309800010 2021-10-25  2635                     HY-VEE #4 / DAVENPORT   
2  INV-41325400013 2021-10-25  4269                       TIPTON FAMILY FOODS   
3  INV-41141800017 2021-10-19  2704               HY-VEE OELWEIN DOLLAR FRESH   
4  INV-41308200068 2021-10-25  2512       HY-VEE WINE AND SPIRITS / IOWA CITY   

                address        city zipcode  \
0        2630 SUMMER ST  BURLINGTON   52601   
1        4064 E 53RD ST   DAVENPORT   52807   
2          610 CEDAR ST      TIPTON   52772   
3  1345 S FREDERICK AVE     OELWEIN   50662   
4    1720 WATERFRONT DR   IOWA CITY   52240   

                                      store_location county_number  \
0  {'type': 'Point', 'coordinates': [-91.117485, ...            29   
1  {'type': 'Point', 'coordinates': [-90.513769, ...            82   
2  {'type': 'Point',

In [9]:
# Drop unnecessary columns
columns_to_drop = [col for col in df.columns if col.startswith('computed_region')]
df.drop(columns=columns_to_drop, inplace=True)

# 3. Handle missing values 
df = df.dropna(subset=['date'])  # Drop rows with missing 'date'
df[['sale_dollars', 'sale_bottles', 'sale_gallons']] = df[['sale_dollars', 'sale_bottles', 'sale_gallons']].fillna(0)

# 4. Save the cleaned data to a new CSV file
df.to_csv('final_cleaned_iowa_liquor_sales.csv', index=False)

# Print out the first few rows to verify
print(df.head())

   invoice_line_no       date store                                      name  \
0  INV-41268900020 2021-10-22  5751  CASEY'S GENERAL STORE #2346 / BURLINGTON   
1  INV-41309800010 2021-10-25  2635                     HY-VEE #4 / DAVENPORT   
2  INV-41325400013 2021-10-25  4269                       TIPTON FAMILY FOODS   
3  INV-41141800017 2021-10-19  2704               HY-VEE OELWEIN DOLLAR FRESH   
4  INV-41308200068 2021-10-25  2512       HY-VEE WINE AND SPIRITS / IOWA CITY   

                address        city zipcode  \
0        2630 SUMMER ST  BURLINGTON   52601   
1        4064 E 53RD ST   DAVENPORT   52807   
2          610 CEDAR ST      TIPTON   52772   
3  1345 S FREDERICK AVE     OELWEIN   50662   
4    1720 WATERFRONT DR   IOWA CITY   52240   

                                      store_location county_number  \
0  {'type': 'Point', 'coordinates': [-91.117485, ...            29   
1  {'type': 'Point', 'coordinates': [-90.513769, ...            82   
2  {'type': 'Point',

In [10]:
import os

# Check the current working directory
current_directory = os.getcwd()
print(f"Current working directory: {current_directory}")


Current working directory: C:\Users\andre
